In [ ]:
# Data representation
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as gph
from yellowbrick.cluster import SilhouetteVisualizer
import graphviz

# Machine learning
import scipy.cluster.hierarchy as ch
from scipy.spatial.distance import cdist

from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, LabelEncoder, OrdinalEncoder

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import  KMeans, MeanShift, estimate_bandwidth, AgglomerativeClustering
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow import keras

# Train and Testing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, silhouette_score, davies_bouldin_score

# Save and get model(s)
import pickle

# Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

## Introduction

In this project, we will apply different stastistical methods, to try and analyse patterns and correlation in different lifestyle factors. This will involve trying to train different machine learning models, to see how precisely these patterns can be used to predict how much carbon is emitted based on the correlating factors.
We will try to answer the following questions: 

- Can an individuals carbon emissions be predicted by their lifestyle factors?
- Which lifestyle factor have the biggest impact on carbon emissions?
- How can this information be used by individuals and the society for reducing carbon emissions?

#### We have formulated the following hypothesises, to help us answer these questions:

H0 = an individuals carbon emissions have no relation to their lifestyle factors

H1 = an individuals carbon emissions can be predicted by their lifestyle factors

H2 = The most important factor for an individuals carbon emission is their style of transport

H3 = The information have clear tendencies, and can make it easy for people to understand how their lifestyle
affects their carbon emissions

H4 = Society can rely on the accuracy of this information for encouraging and/or discouraging behaviours of
individuals

H5 = Time spent on the internet has a significant impact on an individuals carbon emissions

# Data Preparation

### Data Collection / loading:

In [ ]:
# Reading the data from csv into pandas dataframe:
df = pd.read_csv('./data/Carbon-Emission.csv', index_col=None, na_values=['NA'])

In [ ]:
# Viewing the size of the dataframe:
df.shape

In [ ]:
df.sample(5)

In [ ]:
df.iloc[[7511]]

In [ ]:
list(df)

In [ ]:
# Checking for missing values:
df.info()

In [ ]:
# Replacing nan values with None, this we do beacuse we know/can see that pepole that dont drive has nan values as vehicle type
df = df.replace(np.nan, 'None')

In [ ]:
# Verifying that missing values have been replaced:
df.isnull().sum()

In [ ]:
# Getting some insights of the value scope:
df.describe()

In [ ]:
# Chek numeric values for distribution
num_cols = df.select_dtypes(include=['int64']).columns
df_nums = df[num_cols]
df_nums.dtypes

In [ ]:
# Plotting the distribution of the numeric values:
df_nums.hist(figsize=(10, 10), bins=50,)

In [ ]:
# Viewing the distribution of the target variable:
df_nums['CarbonEmission'].hist()

In [ ]:
# Frequency chart for columns:
selected_columns = df.select_dtypes(include=['object']).columns[: -2]

frequency_count_list = {}

for col in selected_columns:
    counts = df[col].value_counts()
    frequency_count_list[col] = counts

for column, counts in frequency_count_list.items(): 
    plt.figure(figsize=(8, 6))
    counts.plot(kind='bar')
    plt.title(f"Frequency chart for column: {column}")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45)

    for i, v in enumerate(counts):
        plt.text(i, v, str(v), ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

### Data cleaning and exploration

It looks like it follows a kind of skewed normal distribution, with a few outliers lying greatly above the mean

In [ ]:
# Encode/transform data numeric
encoder = OrdinalEncoder(dtype=np.int64)

# Making copy of df to a new dataframe called: df_numeric 
df_numeric = df.copy()

# Gettning all columns that has object type:
cate_columns = df.select_dtypes(include=['object']).columns

df_numeric[cate_columns] = encoder.fit_transform(df[cate_columns])

In [ ]:
# Verifying that cols have been encoded:
df_numeric.sample(5)

In [ ]:
print(cate_columns)

In [ ]:
df_numeric.dtypes

In [ ]:
# save Label encoding
with open('models/encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)


In [ ]:
corr_matrix = df_numeric.corr()

In [ ]:
# Heatmap that can show correlation
plt.figure(figsize=(10,6))
sns.heatmap(corr_matrix, annot=True, fmt=".1f")
plt.show()

The biggest correlation with carbon emissions, seem to be the monthly travelled distance by personal vehicle. Other factor with clear correlations seem to be type of transport, vehicle type and frequency of travelling with airplane. There also seem to be a smaller correlation with the amount of clothes people buy.

In [ ]:
plt.xlabel('Vehicle Monthly Distance Km')
plt.ylabel('Carbon Emissions')
plt.scatter(df_numeric['Vehicle Monthly Distance Km'], df_numeric['CarbonEmission'])
plt.show()

Does internet usage have an impact on carbon emissions?

In [ ]:
corr_matrix['CarbonEmission']['How Long Internet Daily Hour']

In [ ]:
plt.xlabel('How Long Internet Daily Hour')
plt.ylabel('Carbon Emissions')
plt.bar(df_numeric['How Long Internet Daily Hour'], df_numeric['CarbonEmission'])
plt.show()

There is a slight correlation, but nowhere near enough to asser any tendencies. We are lucky computer people!

Let's see if theres as difference between male an female patterns

In [ ]:
# Split dataframe based on sex, and see if theres a difference in the correlation
df_male = df_numeric[df_numeric['Sex'] == 1]
df_male

In [ ]:
df_female = df_numeric[df_numeric['Sex'] == 0]
df_female

In [ ]:
df_male_corr = df_male.corr()
df_female_corr = df_female.corr()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df_male_corr, annot=True, fmt=".1f")
plt.title('Male correlation')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df_female_corr, annot=True, fmt=".1f")
plt.title('Female correlation')
plt.show()

There doesn't really seem to be a difference of how much the factors weigh between males and females

### Outliers

In [ ]:
df_numeric.plot.box(rot=90, figsize=(10,6))

In [ ]:
df_nums['Vehicle Monthly Distance Km'].plot.box()

In [ ]:
med = df_nums['Vehicle Monthly Distance Km'].median()
df_nums['Vehicle Monthly Distance Km'].max() - med

As we can see, the distribution of distance travelled by vehicle is very skewed, with 75% of the data lying beneath 3000 km pr month, with quite a few outlier being outragerously above the median. This might still be an accurate representation though, since some individuals just might travel that much. 
We'll keep it for the variance

Let's see if people are as accurate as they think, when they say they're energy efficient!

In [ ]:
test = df_numeric.iloc[[6550]]
test

In [ ]:
grouped_data = df.groupby('Energy efficiency')

grouped = df.groupby('Energy efficiency')['CarbonEmission'].agg(['mean', 'sum']).reset_index()
grouped

In [ ]:
plt.figure(figsize=(8, 6))

# Define the width of each bar
bar_width = 0.35

# Define the positions for the bars
index = np.arange(len(grouped['Energy efficiency']))

# Create side-by-side bars for 'Sum' and 'Mean'
plt.bar(index, grouped['mean'], bar_width, color='red')

# Add labels, title, and legend
plt.xlabel('Category')
plt.ylabel('Mean Carbon Emissions')
plt.title('Mean by Energy efficiency')
plt.xticks(index + bar_width / 2, grouped['Energy efficiency'])
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
grouped['sum'] = grouped['sum'].astype(float)
grouped.plot.bar(x='Energy efficiency', y='sum', figsize=(8, 6), rot=0, title='Sum by Energy efficiency (values in millions)')

We can see that people say that the are energy efficient generally tend to have a little bit lower carbon emission in total, but the difference really is only about 1,5% lower than the 'No' category.

The total sum of carbon emissions seems to be higher by people declaring that they're being energy efficient, indicating that a lot more people think they are being efficient than not, while not really making any difference in the emitted carbon dioxide!

### Plane travel

We can see theres a small correlation between the Frequency of air travel, and their carbon emission, let's sxplore what it means

In [ ]:
group_air = df.groupby('Frequency of Traveling by Air').size().reset_index(name='Count')
group_air

In [ ]:
# Plot the distribution of the frequency
group_air.plot.pie(y='Count', labels=['Never', 'Rare', 'Frequently', 'Very Frequently'], autopct='%1.1f%%', figsize=(8, 8))

In [ ]:
grouped = df.groupby('Frequency of Traveling by Air')['CarbonEmission'].agg(['mean', 'sum']).reset_index()
grouped

Total mean = 2262.8

total mean / very frequently = 0.75

Very frequently travelers emit around 25% on average than the average across all flight travel categories

In [ ]:
grouped.plot.bar(x='Frequency of Traveling by Air', y=['mean'], figsize=(8, 6), rot=0)

As we can see, the distribution of people is fairly equal between all different categories of flight frequency, but there's a clearly tendency of higher average carbon emissions from people who are in the 'very frequent' category.

The mean carbon emssions are around 22% higher for people who travel very frequently by air.


## Supervised Machine learning

Let's try and see if we can predict total carbon emission values by training different classifier models.
Since we're doing classification, we'll bin the carbon emission numbers into categories

In [ ]:
df_numeric.sample(5)

In [ ]:
df_numeric.columns

In [ ]:
scaled_data = df_numeric[['CarbonEmission']]

In [ ]:
qtrans = QuantileTransformer(output_distribution='normal', random_state=0)
scaled_data['CarbonEmission_trans_norm'] = qtrans.fit_transform(scaled_data[['CarbonEmission']])

print('Mean:', scaled_data['CarbonEmission_trans_norm'].mean())
print('Standard Deviation:', scaled_data['CarbonEmission_trans_norm'].std())

In [ ]:
scaled_data.describe()

In [ ]:
df_part = df_numeric.copy().drop(columns=['CarbonEmission'], axis=1)
df_bin = pd.concat([df_part, scaled_data['CarbonEmission_trans_norm']], axis=1)
df_bin

In [ ]:
emission_max = df_bin['CarbonEmission_trans_norm'].max()
emission_min = df_bin['CarbonEmission_trans_norm'].min()

bins = np.linspace(emission_min -1, emission_max, 5)

df_bin['EmissionBin'] = pd.qcut(df_bin['CarbonEmission_trans_norm'], q=4, labels=False)
df_bin.describe()


In [ ]:
grouped_bin = df_bin.groupby('EmissionBin').size().reset_index(name='Count')
grouped_bin

In [ ]:
df_bin.drop('CarbonEmission_trans_norm', axis=1, inplace=True)
df_bin.sample(5)

## Prediction by classification

#### Naive Bayes model

In [ ]:
df_bin.head()

In [ ]:
# Convert the dataset into array
array = df_bin.values
# Create two axis for the test data
X = array[:,0:19] 
Y = array[:,-1]   # CarbonEmission
Y

In [ ]:
# split proportion
test_set_size = 0.15

# Initial value for randomization
seed = 42 # this is the answer
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_set_size, random_state=seed)

In [ ]:
# Use GaussianNB for numeric data
nb = GaussianNB()
nb.fit(X_train, Y_train)

In [ ]:
# test the model
nb.score(X_test, Y_test)

An accuracy of about 70 percent, not bad for a first try, but I think we can do better.
Let's try to focus on the correlating values

In [ ]:
column_indexes = ['Body Type', 'Diet', 'How Often Shower', 'Heating Energy Source',
       'Transport', 'Social Activity', 'Monthly Grocery Bill',
       'Frequency of Traveling by Air', 'Vehicle Monthly Distance Km',
       'Waste Bag Size', 'Waste Bag Weekly Count',
       'How Many New Clothes Monthly'] # Columns with correlation > 0.1
df_features = df_numeric[column_indexes]
df_features.columns

In [ ]:
X = df_numeric[column_indexes].values
X


In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_set_size, random_state=seed)
Y_test.shape

In [ ]:
nb = GaussianNB()
nb.fit(X_train, Y_train)

#### Validation

In [ ]:
prediction = nb.predict(X_test)
prediction

In [ ]:
print(accuracy_score(Y_test, prediction))

In [ ]:
cmat = confusion_matrix(Y_test, prediction)
print(cmat)
print(classification_report(Y_test, prediction))

In the end, it made a bigger difference to adjust the test set size to around 15% of the set

### Decision Tree Classifier

In [ ]:
test_set_size = 0.15
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_set_size, random_state=seed)
Y_test.shape

In [ ]:
# Build Decision Trees Classifier 
params = {'max_depth': 6}
treeM = DecisionTreeClassifier(**params)
treeM.fit(X_train, Y_train)

In [ ]:
gr_data = tree.export_graphviz(treeM, out_file=None, 
                         feature_names=df_features.columns, class_names = True,        
                         filled=True, rounded=True, proportion = False, special_characters=True)  
dtree = graphviz.Source(gr_data)

In [ ]:
# render the tree as pdf
dtree.render('dtree_render', format='pdf', cleanup=True)

#### Validation

In [ ]:
y_predict = treeM.predict(X_test)
y_predict

In [ ]:
cmat = confusion_matrix(Y_test, y_predict)
print(cmat)
print(classification_report(Y_test, y_predict))

Even better than before. Maybe our predictions can be even more accurate by using several decision trees

### Random Forest Classifier

In [ ]:
test_set_size = 0.15
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_set_size, random_state=seed)
Y_test.shape

In [ ]:
forest = RandomForestClassifier(n_estimators = 300, max_depth = 13)
forest.fit(X_train, Y_train)

#### Validation

In [ ]:
y_predict = forest.predict(X_test)
y_predict

In [ ]:
cmat = confusion_matrix(Y_test, y_predict)
print(cmat)
print(classification_report(Y_test, y_predict))

In [ ]:
pickle.dump(forest, open('./models/forest_model.pkl', 'wb'))

### Classification summary

**Iteration 1**
- No data normalization
- 5 equally divided bins <br />

80% accuracy (RFC)

At 80% accuracy the random forest classifier seems to be the most accurate model we have tested. It seems to correctly categorize a lot of the lower emitting categories, and is a little less accurate on cases with fewer instances. It's still making some big mistakes which might not show up in the overall calculated score, but is very significant.
For example, none of the models have succesfully caught any people in the topmost carbon emitting category. Perhaps implying, that the category data sould be binned differently, becuase if only very few instances fit into the bin, the model might not have any chance of recieving enough training data to be able to recognize the extreme cases.

**Iteration 2**
- Data transformed to normal distribution
- 4 equally divided bins

87% accuracy (RFC).

Model accuracy have improved in accuracy. I accredit this to the fewer bins and the now normal distribution of the data. Even though the accuracy seems higher, the result is still unsatisfactory since it fails to identify any extreme cases in both high and low end. Because of the normal distribution transform, the data have accumulated around the mean, resulting in only around 2% of the data fitting in the extreme bins. The training data might still be to sparse to let the model identify these few extreme cases

**Iteration 3**
- Data transformed to normal distribution
- 3 equally divided bins

92% accuracy (RFC)

Model have once model improved in accuracy. Even with only 3 bins, the middle bin is responsible for 92% of the data. The fewer amount of bins, gives better results when identifying outlier cases, but it is still very overfitted and not very useful

**Iteration 4**
- Data transformed to normal distribution
- 4 bins divided by quantiles

67% accuracy (RFC)

Accuracy has dropped to 67%, but seems to catch a lot more of the extreme cases, now the bin sizes have been adjusted. It now represents the data tendencies way better, and are less overfitted.



## Prediction by regression

### XGBregressor

In [ ]:
# Selecting the features (x) and the target (y)
ml_x = df_numeric.drop('CarbonEmission', axis=1).values
ml_y = df_numeric['CarbonEmission'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ml_x, ml_y, test_size=0.2, random_state=1)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Initialize the model(s)
xgbr = XGBRegressor()
linearreg = LinearRegression()
svr = SVR()

In [ ]:
# Fit the model(s)
xgbr.fit(X_train, y_train)
linearreg.fit(X_train, y_train)
svr.fit(X_train, y_train)

In [ ]:
# Predict the target
y_pred_xgbr = xgbr.predict(X_test)
y_pred_linearreg = linearreg.predict(X_test)
y_pred_svr = svr.predict(X_test)

In [ ]:
ml_res = {
    "ML models": ["XGBRegressor", "LinearRegression", "SVR"],
    "r2": [
        r2_score(y_test, y_pred_xgbr),
        r2_score(y_test, y_pred_linearreg),
        r2_score(y_test, y_pred_svr)
    ],
    "MAE": [
        mean_absolute_error(y_test, y_pred_xgbr),
        mean_absolute_error(y_test, y_pred_linearreg),
        mean_absolute_error(y_test, y_pred_svr)
    ]
}
# Create a dataframe
ml_res_df = pd.DataFrame(ml_res)
ml_res_df

The XGBoost model looks to have the best accuracy, with a score of 0.9747 on the test set, with only a MAE of 123.98. 

In [ ]:
# Storing xgboost model
pickle.dump(xgbr, open('./models/xgboost_model.pkl', 'wb'))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test, y_pred_xgbr, 'o')
plt.plot(y_test, y_test, 'r')
plt.title("XGBRegressor")
plt.xlabel("Actual values")
plt.ylabel("Predicted values")
plt.show()

### Neural Network

In [ ]:
ml_x = df_numeric.drop('CarbonEmission', axis=1).columns
ml_x

In [ ]:
ml_x = df_numeric.drop('CarbonEmission', axis=1).values
ml_y = df_numeric['CarbonEmission'].values

scaler = StandardScaler()
y = ml_y
x = scaler.fit_transform(ml_x)
print(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
# Save scaler
with open('models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
# Create model for neural network:
# Selecting input shape, and defining number of layers & neurons
model = keras.Sequential([
    keras.layers.Input(shape=(19,)),             # 19 feature / input neurons
    
    keras.layers.Dense(256, activation='relu'),  
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),

    keras.layers.Dense(1)                        # Output layers
])

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=50)

In [ ]:
t_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {t_loss:.2f}")

In [ ]:
pred = model.predict(X_test)
print(r2_score(y_test, pred))

In [ ]:
pred

In [ ]:
pickle.dump(model, open('./models/nn_model.pkl', 'wb')) 

### Neural Network Results

 [x1,... xn] means the numbers of neurons for each layer in the network

<table><tr>
<td>

|Layers                      | Epochs | Accuracy | Test Loss | MAE     |
|----------------------------| -------| -------- | ----------| --------|
|1 lay [32]                  | 50     | 0.6514   | 485.4101  | 488.3452|
|1 lay [64]                  | 50     | 0.6794   | 458.6350  | 457.2778|
|1 lay [128]                 | 50     | 0.7634   | 360.0965  | 358.2312|
|1 lay [256]                 | 50     | 0.8073   | 314.0002  | 303.8010|
|1 lay [512]                 | 50     | 0.8387   | 277.7811  | 267.3893|


</td><td>

|Layers                      | Epochs | Accuracy | Test Loss | MAE     |
|----------------------------| -------| -------- | ----------| --------|
|2 lay [32, 64]              | 50     | 0.8260   | 305.7189  | 287.4525|
|2 lay [64, 128]             | 50     | 0.8519   | 280.6377  | 259.1389|
|2 lay [128, 256]            | 50     | 0.9222   | 195.7561  | 177.1057|
|2 lay [256, 512]            | 50     | 0.9588   | 154.8777  | 133.3263|

</td></tr>

<tr>
<td>

|Layers                      | Epochs | Accuracy | Test Loss | MAE     |
|----------------------------| -------| -------- | ----------| --------|
|3 lay [32, 64, 128]         | 50     | 0.8919   | 219.7377  | 210.3670|
|3 lay [64, 128, 256]        | 50     | 0.9469   | 167.5058  | 152.9221|
|3 lay [128, 256, 512]       | 50     | 0.9755   | 126.4031  | 104.1510|
|3 lay [256, 512, 512]       | 50     | 0.9800   | 115.4065  | 81.7558 |

</td><td>

|Layers                      | Epochs | Accuracy | Test Loss | MAE     |
|----------------------------| -------| -------- | ----------| --------|
|4 lay [32, 64, 128, 256]    | 50     | 0.9787   | 117.9240  | 108.2563|
|4 lay [64, 128, 256, 512]   | 50     | 0.9792   | 117.7210  | 94.6650 |
|4 lay [128, 256, 512, 512]  | 50     | 0.9713   | 134.8475  | 80.8660 |
|4 lay [256, 512, 512, 512]  | 50     | 0.9819   | 109.2275  | 75.5905 |

</td></tr></table>

<!-- |19 lay [all 32] | 50     | 0.974    | 126.68 | v| -->

NB: Source used for helping to make this neural network: [Neural Network Regression Implementation and Visualization in Python](https://medium.com/@nandiniverma78988/neural-network-regression-implementation-and-visualization-in-python-d5893713ed79)

#### Approach (simple model)

The approach was to start with a simple model, and then add complexity over time to try to increase the accuracy of the model. At the beginning, we chose to start with only one hidden layer and 32 neurons, and then each time increase neurons times 2. We made the assumption that the max number of neurons in a layer should be 512 to avoid overfitting, beacuse we know that more neurons means that the model has a higher capacity to learn form the traning data; however we dont want the model to learn from noise and other irrelevant factors, which can lead to overfitting.

#### Configuration

The 4 performance metrics of the neural network with varying configurations over 50 epochs of layers and neurons are shown in the table above. This config are defined by the number of layers and the number of neurons in each layer. For all iterations, we chose the activations function: ReLU:
```
f(x) = max(0, x)
```

#### Results

The results show that the accuracy of the model increases with the number of layers and neurons. We can see that the accuracy of the model is 0.9819 with 4 layers (256, 512, 512, 512 neurons) The test loss is 109.2275 and the mean absolute error is 75.5905. This is the best result we have achieved with the neural network.
We can also conclude that the neural network has better results the our XGBoost model, that 'only' got an accuracy of 0.9747 and a MAE of 123.9864.


#### Alterantive improvements

Without a doubt, we know that our neural network has a good accuracy 0.9819 and a littel MAE on 75.5905. However, we can still alot of different was to try and improve the model.
The different tings that can be experimaneted with are the following:
- The number of epochs
- The number of layers
- The number of neurons in each layer
- The activation function

## Unsupervised machine learning

### Applying the data to K-means, Hierarchical clustering & Mean Shift

The perpose of applying these clustering models, is to see if, there are patterns in the data, that we haven't been able to see before.

In [ ]:
scaler = StandardScaler()

# Create an instance of PCA
pca = PCA(n_components=2)

X_scaled = scaler.fit_transform(df_numeric)

# Fit the PCA model to the data
X_pca = pca.fit_transform(X_scaled)

# Create a new DataFrame with the PCA results
df_numeric_pca = pd.DataFrame(X_pca, columns=[f'PC{i}' for i in range(1, pca.n_components_ + 1)])
df_numeric_pca


In [ ]:
# Create a scatter plot of the first two principal components
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Scatter Plot')
plt.show()


In [ ]:
X = df_numeric_pca.values
X

### Hierarchical clustering

In [ ]:
# Making the dendrogram to find the optimal number of clusters
plt.figure(figsize=(20,10))

dendogram = ch.dendrogram(ch.linkage(X, method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('Observations')
plt.ylabel('Euclidean distance')
plt.show()

In [ ]:
# Hierarchical Clustering Model
n_clusters = 2
model = AgglomerativeClustering(n_clusters, affinity = 'euclidean', linkage = 'ward')
model.fit(X)

In [ ]:
Y = model.fit_predict(X)
Y

In [ ]:
# Visualising the clusters
plt.scatter(X[:, 0], X[:, 1], s=30, c=Y, cmap='viridis')
plt.title('Discovered Clusters')
plt.xlabel('X1')
plt.ylabel('X2')
# plt.legend()
plt.show()

In [ ]:
hc = model.fit_predict(X)
print("Hierarchical Silhouette Score:", silhouette_score(X, hc))

### K-means

In [ ]:
# KMeans Clustering Model
# Extracting the features
X_kmeans = X
X_kmeans

In [ ]:
# Using the elbow method to find the optimal number of clusters
distortions = []
K = range(2,10)
for k in K:
    model = KMeans(n_clusters=k, n_init=10).fit(X_kmeans)
    model.fit(X_kmeans)
    distortions.append(sum(np.min(cdist(X_kmeans, model.cluster_centers_, 'euclidean'), axis=1)) / X_kmeans.shape[0]) 
print("Distortion: ", distortions)

In [ ]:
# Plotting the elbow method
plt.title('Elbow Method for Optimal K')
plt.plot(K, distortions, 'bx-')
plt.xlabel('K')
plt.ylabel('Distortion')
plt.show()

In [ ]:
k_clusters = 2

In [ ]:
kmeans = KMeans(init='k-means++', n_clusters=k_clusters, n_init=20)

In [ ]:
kmeans.fit(X_kmeans)

In [ ]:
# Silhouette score for KMeans clustering for different number of clusters
scores = []
K = range(2, 10)
for k in K:
    model = KMeans(n_clusters=k, n_init=10)
    model.fit(X_kmeans)
    score = metrics.silhouette_score(X_kmeans, model.labels_, metric='euclidean', sample_size=len(X_kmeans))
    print("\nNumber of clusters =", k)
    print("Silhouette score =", score)
    scores.append(score)

In [ ]:
# Plot the elbow
plt.title('Silhouette Score Method for Discovering the Optimal K')
plt.plot(K, scores, 'bx-')
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
y = kmeans.predict(X_kmeans)
y

In [ ]:
print(kmeans.labels_)

In [ ]:
# Visualising the endividual clusters
for i in range(k_clusters):
    # slice the cluster
    cluster = X_kmeans[y == i]    
    # print the shape
    print("Cluster ", i, ": ", cluster.shape)    
    # plot the points of this cluster
    plt.scatter(cluster[:, 0], cluster[:, 1])   
    plt.grid(True)
    plt.show()

In [ ]:
# Visualising all the clusters
plt.scatter(X_kmeans[:, 0], X_kmeans[:, 1], c=y, s=20, cmap='viridis')
plt.grid(True)
plt.show()

In [ ]:
print(kmeans.cluster_centers_)

In [ ]:
# Visualizing the Silhouette Score
visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
visualizer.fit(X_kmeans)
visualizer.show()  

### Mean Shift

In [ ]:
# Extracting the features
x_ms = X

In [ ]:
# Estimate the bandwidth of the data
bandwidth = estimate_bandwidth(x_ms, quantile=0.2, n_samples=200)
bandwidth 

In [ ]:
# Making the instance of the MeanShift model
msmodel = MeanShift(bandwidth=bandwidth, bin_seeding=True)
msmodel.fit(x_ms)

In [ ]:
# Getting the labels
labels = msmodel.labels_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
n_clusters_

In [ ]:
cluster_centers = msmodel.cluster_centers_
cluster_centers
# Predict the cluster for all the samples
Y = msmodel.predict(x_ms)
Y

In [ ]:
len(msmodel.labels_)

plt.scatter(x_ms[:,0], x_ms[:,1], s=50, c=labels, cmap='viridis')
plt.title(f'Estimated number of clusters = {n_clusters_}')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Visualising the clusters in 3D
df_3d = pd.DataFrame(x_ms, columns=['x', 'y'])
df_3d['label'] = labels

# Create a 3D scatter plot
fig = px.scatter_3d(df_3d, x='x', y='y', z='label', color='label', symbol='label')
fig.show()

fig = plt.figure()
plt.title('Discovered Clusters')
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_ms[:,0], x_ms[:,1],  marker='o', cmap='viridis', c=labels)
ax.scatter(cluster_centers[:,0], cluster_centers[:,1], marker='x', 
           color='red', s=100, linewidth=3, zorder=10)
plt.show()

In [ ]:
# Silhouette score for MeanShift clustering
score = silhouette_score(x_ms, msmodel.labels_, metric='euclidean')
print('Silhouette Score: %.3f' % score)

## Conclusion

Theres much interesing information in exploring this dataset, and we can definetely find a lot of patterns that is crucial for identifying which personal lifestyle factors are significant when it comes to calculationg our total carbon emission output.

**Here is our discoveries summarized:**

- The most significant factors for an individuals carbon emission seem to be their choice of transport. People that drive many kilometers monthly in personal vehicles are also the one who have a tendency to have the highest carbon emissions.

- Theres a small indication that internet usage has an impact on carbon emissions to correlating factors, but no direct pattern can be concluded, so we deem it as insignificant

- There seem to be very little difference in the correlating factors between males and females

- People who see themselves as 'Energy efficient' only seem the emit about 1,5% less carbon than people who say they're 'Not energy efficient', which we conclude makes it insignificant

- People who travel with airplanes very frequently seem to emit around 22% more carbon on average, than people who never fly

#### Prediction by classification
We tried to use different classification machine learning methods to try and predict carbon output based on correlating factors, and clustering methods for trying to discov hidden patterns in the data.

For classification we tried:

- Naive Bayes classifier
- Decision Tree classifier
- Random Forest classifier

We tried several iterations for optimizing the models by adjusting the category amount and the normalisation of the input data. The RFC model was the most succesful, with an accuracy of aaround 92% with 3 categories. However, we deem the result as unsaticfactory, since the input data still is a bit unsuited for this kind of machine learning, with 92% of the data fitting into a single category. The good accuracy is due to overfitting.

On the last iteration we tried to divide the categories by quantiles, which made the model drop to 67% accuracy, but is a way more useful and less overfitted model.
We therefore conclude the best classification model to be the RFC model with training data binned into 4 categories based on the quartiles of the data, since it could make decently accurate predictions, and represent the tendencies in the data without being overfitted to the training data. However, this kind of model is definetely unsuited for this task. Even though we got satisfactory predictions, the information value of the predictions are very low, since the binning of the results into categories, makes the prediction results vague and without meaning.

#### Prediction by regression

Since we are trying to predict numeric values, we thought might prove more valuable to try and predict actual carbon emission values by using regression models.
This turned out to be a way better fit than the classification models, both in accuracy and in the information value of the output.

we used an advanced gradient boosting model for this purpose
in the form of an XGBRegressor.

It was quick to train the model, and
we ended up getting very good results,
wtih an accuracy of 97% on the test set.

A good fit for this data.

#### Clustering summary:

We have try using K-means, Hierarchical clustering & Mean shift to see if there are any patterns in the data, but it seems that there are no clear patterns in the data, when applying these clustering methods to the hole dataset. We have tried with and without scaling the data. Additionally, we have tried to use different numbers of clusters, but the results are still the same. 
This is concluded based on the visualisation of the clusters from the different methods due to the visualistations being poor; However, the silhouette score have actually been quite high, which indicates that there are some patterns in the data.

Silhouette score (without scaling and pca):
* K-means: 0.727 (with 2 clusters), 0.624 (with 3 clusters)
* Hierarchical clustering: 0.426
* Mean shift: 0.727

When applying scaling and pca to the data, the visualisation of the clusters are much better; however, the silhouette score is lower than without scaling and pca:

Silhouette score (with scaling and pca):
* K-means: 0.549 (2 clusters), 0.426 (3 clusters)
* Hierarchical clustering: 0.542 (2 clusters)
* Mean shift: 0.546 (2 clusters)

We have also tried using only some parts of the datas features, these results are gives som visualisation endication of patterns.
The features used for the clustering are: 'Monthly Grocery Bill', 'Waste Bag Weekly Count', 'How Long TV PC Daily Hour', 'How Many New Clothes Monthly', 'How Long Internet Daily Hour'

Silhouette score (without scaling and pca):
* K-means: 0.79 (with 2 clusters), 0.727 (with 3 clusters)
* Hierarchical clustering: 0.698
* Mean shift: 0.597

Other features combinations have also been tried, but the results gives similar visualisation and silhouette score.